In [1]:
#!pip uninstall accelerate peft bitsandbytes transformers trl -y
#!pip install accelerate peft==0.13.2 bitsandbytes transformers trl==0.12.0
#!pip install huggingface_hub datasets
#!pip install fsspec==2023.9.2
#!pip install --upgrade datasets
import os
from datasets import config
import shutil
cache_dir = config.HF_DATASETS_CACHE
if os.path.exists(cache_dir):
    shutil.rmtree(cache_dir)
import torch
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import os
import re
import gc
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline as hf_pipeline
)

# Add device detection and configuration optimized for Kaggle
def get_device_config():
    """Check available hardware and return appropriate configuration for Kaggle"""
    try:
        # Force garbage collection to free up memory
        gc.collect()
        torch.cuda.empty_cache()
        
        if torch.cuda.is_available():
            # Get GPU memory info
            gpu_memory = torch.cuda.get_device_properties(0).total_memory
            free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
            
            print(f"CUDA available: {torch.cuda.get_device_name(0)}")
            print(f"Total GPU memory: {gpu_memory / 1e9:.2f} GB")
            print(f"Free GPU memory: {free_memory / 1e9:.2f} GB")
            
            # Determine quantization based on available memory
            if free_memory > 4e9:  # More than 4GB free
                return {
                    "device_map": "auto",
                    "use_cuda": True,
                    "load_in_4bit": True,
                    "bnb_4bit_compute_dtype": torch.float16,
                    "bnb_4bit_quant_type": "nf4"
                }
            else:
                # Lower precision for smaller memory
                return {
                    "device_map": "auto",
                    "use_cuda": True,
                    "load_in_4bit": True,
                    "bnb_4bit_compute_dtype": torch.float16,
                    "bnb_4bit_quant_type": "nf4",
                    "low_cpu_mem_usage": True
                }
        else:
            print("CUDA not available, falling back to CPU")
            return {
                "device_map": "cpu",
                "use_cuda": False,
                "load_in_4bit": False  # Disable 4-bit quantization on CPU
            }
    except RuntimeError as e:
        print(f"CUDA initialization error: {e}")
        print("Falling back to CPU")
        return {
            "device_map": "cpu",
            "use_cuda": False,
            "load_in_4bit": False
        }

# Modified model loading function with Kaggle-specific optimizations
def load_model(model_path="aboonaji/llama2finetune-v2"):
    """Load model with appropriate configuration for Kaggle GPU"""
    try:
        # Get device configuration
        config = get_device_config()
        
        # Handle Kaggle-specific auth token for Hugging Face
        hf_token = os.environ.get('HUGGINGFACE_TOKEN', None)
        token_kwargs = {"token": hf_token} if hf_token else {}
        
        # Try to configure tokenizer with proper error handling
        try:
            tokenizer = AutoTokenizer.from_pretrained(
                pretrained_model_name_or_path=model_path,
                trust_remote_code=True,
                **token_kwargs
            )
            tokenizer.pad_token = tokenizer.eos_token
            tokenizer.padding_side = "right"
        except Exception as e:
            print(f"Error loading tokenizer: {e}")
            print("Attempting to download tokenizer with use_auth_token")
            try:
                # Alternative loading approach for Kaggle
                tokenizer = AutoTokenizer.from_pretrained(
                    pretrained_model_name_or_path=model_path,
                    use_auth_token=True if hf_token else False,
                    trust_remote_code=True
                )
                tokenizer.pad_token = tokenizer.eos_token
                tokenizer.padding_side = "right"
            except Exception as e2:
                print(f"Second attempt at loading tokenizer failed: {e2}")
                return None, None

        # Configure model based on available hardware
        try:
            if config["use_cuda"]:
                # Before loading model, ensure maximum memory is available
                gc.collect()
                torch.cuda.empty_cache()
                
                # GPU configuration with BitsAndBytes
                quantization_config = BitsAndBytesConfig(
                    load_in_4bit=True,
                    bnb_4bit_compute_dtype=torch.float16,
                    bnb_4bit_quant_type="nf4"
                )

                model = AutoModelForCausalLM.from_pretrained(
                    pretrained_model_name_or_path=model_path,
                    quantization_config=quantization_config,
                    device_map="auto",
                    low_cpu_mem_usage=True,  # Important for Kaggle
                    **token_kwargs
                )
                model.gradient_checkpointing_enable()
                model.config.use_cache = False
                model.config.pretraining_tp = 1

            else:
                # CPU configuration without quantization
                model = AutoModelForCausalLM.from_pretrained(
                    pretrained_model_name_or_path=model_path,
                    device_map="cpu",
                    torch_dtype=torch.float32,  # Use full precision on CPU
                    low_cpu_mem_usage=True,
                    **token_kwargs
                )

            return model, tokenizer
            
        except Exception as e:
            print(f"Error loading model: {e}")
            # Try alternative loading method for Kaggle
            try:
                print("Attempting alternative model loading approach...")
                if config["use_cuda"]:
                    model = AutoModelForCausalLM.from_pretrained(
                        pretrained_model_name_or_path=model_path,
                        device_map="auto",
                        torch_dtype=torch.float16,  # Use half precision as fallback
                        use_auth_token=True if hf_token else False,
                        trust_remote_code=True
                    )
                else:
                    model = AutoModelForCausalLM.from_pretrained(
                        pretrained_model_name_or_path=model_path,
                        device_map="cpu",
                        use_auth_token=True if hf_token else False,
                        trust_remote_code=True
                    )
                return model, tokenizer
            except Exception as e2:
                print(f"Alternative model loading also failed: {e2}")
                return None, None

    except Exception as e:
        print(f"Unexpected error in load_model: {e}")
        return None, None

# Modified EmotionalSupportBot with memory optimizations
class EmotionalSupportBot:
    def __init__(self, model=None, tokenizer=None, cpu_fallback=True):
        """
        Initialize the EmotionalSupportBot with model and tokenizer

        Args:
            model: Pre-loaded model or None to load default
            tokenizer: Pre-loaded tokenizer or None to load default
            cpu_fallback: Whether to fall back to dummy mode if model loading fails
        """
        self.model = model
        self.tokenizer = tokenizer
        self.is_dummy_mode = False

        # If model or tokenizer not provided, try to load them
        if model is None or tokenizer is None:
            print("Loading model and tokenizer...")
            try:
                self.model, self.tokenizer = load_model()

                if self.model is None and cpu_fallback:
                    print("Model loading failed. Falling back to dummy mode.")
                    self._initialize_dummy_mode()
                    return

            except Exception as e:
                print(f"Error initializing model: {e}")
                if cpu_fallback:
                    print("Falling back to dummy mode due to initialization error.")
                    self._initialize_dummy_mode()
                    return
                else:
                    raise

        # Initialize text generation pipeline with memory-optimized settings
        try:
            if not self.is_dummy_mode:
                # Check if CUDA is available and has limited memory
                if torch.cuda.is_available():
                    free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
                    
                    # Adjust max_new_tokens based on available memory
                    max_new_tokens = 128 if free_memory < 2e9 else 256  # Reduced from 256/512
                    print(f"Setting max_new_tokens to {max_new_tokens} based on available GPU memory")
                    
                    self.text_generation_pipeline = hf_pipeline(
                        task="text-generation",
                        model=self.model,
                        tokenizer=self.tokenizer,
                        max_new_tokens=max_new_tokens,
                        truncation=True,
                        do_sample=True,
                        temperature=0.7,
                        top_p=0.9,
                        repetition_penalty=1.2,
                        device_map="auto",
                        pad_token_id=self.tokenizer.eos_token_id
                    )
                else:
                    self.text_generation_pipeline = hf_pipeline(
                        task="text-generation",
                        model=self.model,
                        tokenizer=self.tokenizer,
                        max_new_tokens=128,  # Reduced for CPU
                        truncation=True,
                        do_sample=True,
                        temperature=0.7,
                        top_p=0.9,
                        repetition_penalty=1.2,
                        pad_token_id=self.tokenizer.eos_token_id
                    )
        except Exception as e:
            print(f"Error initializing pipeline: {e}")
            if cpu_fallback:
                print("Falling back to dummy mode due to pipeline error.")
                self._initialize_dummy_mode()
                return
            else:
                raise

        # *** MEMORY OPTIMIZATION: Don't store bot responses ***
        # Conversation memory now only stores user messages with reduced history
        self.last_user_messages = {}  # Dictionary to store only recent user messages by user ID
        self.max_history_length = 2   # Reduced from 3 to 2 to prevent memory issues

        # Emotion tracking
        self.user_emotions = {}  # Store emotion history by user ID

        # Safety guardrails - response templates for critical situations
        self.crisis_keywords = ["suicide", "kill myself", "end my life", "want to die"]
        self.crisis_response = (
            "I notice you've mentioned something that sounds serious. If you're having thoughts of "
            "harming yourself, please know that you're not alone and support is available. "
            "Please consider talking to a mental health professional or calling a crisis helpline:\n"
            "- National Suicide Prevention Lifeline: 988 or 1-800-273-8255\n"
            "- Crisis Text Line: Text HOME to 741741\n\n"
            "Would you like to talk more about what you're experiencing? I'm here to listen."
        )

        print("EmotionalSupportBot initialized successfully.")

    def _initialize_dummy_mode(self):
        """Initialize a dummy mode version of the bot for when model loading fails"""
        print("Initializing EmotionalSupportBot in dummy mode...")
        self.is_dummy_mode = True

        # Set minimal dummy objects
        class DummyModel:
            def generate(self, **kwargs):
                return [[0]]

        class DummyTokenizer:
            eos_token = "</s>"
            pad_token = "</s>"
            padding_side = "right"

            def encode(self, *args, **kwargs):
                return [0]

            def decode(self, *args, **kwargs):
                return ""

        self.model = DummyModel()
        self.tokenizer = DummyTokenizer()

        # Create a custom dummy response function
        def dummy_generate(text):
            # Simple rule-based responses
            text_lower = text.lower()

            if "how are you" in text_lower:
                response = "I'm here to listen and support you. How are you feeling today?"
            elif any(word in text_lower for word in ["sad", "down", "depressed", "unhappy"]):
                response = "I'm sorry to hear you're feeling down. Would you like to talk about what's contributing to these feelings?"
            elif any(word in text_lower for word in ["anxious", "worried", "nervous", "stress"]):
                response = "Anxiety can be really challenging. Would it help to explore what's causing these feelings of worry?"
            elif any(word in text_lower for word in ["angry", "mad", "frustrated"]):
                response = "I can understand feeling frustrated. Sometimes anger points to something important to us. Would you like to discuss what's behind these feelings?"
            elif any(word in text_lower for word in ["alone", "lonely", "isolated"]):
                response = "Feeling alone can be really difficult. Connection is so important for our wellbeing. Have you been able to reach out to anyone recently?"
            else:
                response = "Thank you for sharing that with me. Would you like to tell me more about what you're experiencing?"

            # Format the response to match the expected format
            full_response = text + " [/INST] " + response
            return [{"generated_text": full_response}]

        self.text_generation_pipeline = dummy_generate

        # Initialize conversation memory - OPTIMIZED
        self.last_user_messages = {}
        self.max_history_length = 2

        # Emotion tracking
        self.user_emotions = {}

        # Safety guardrails
        self.crisis_keywords = ["suicide", "kill myself", "end my life", "want to die"]
        self.crisis_response = (
            "I notice you've mentioned something that sounds serious. If you're having thoughts of "
            "harming yourself, please know that you're not alone and support is available. "
            "Please consider talking to a mental health professional or calling a crisis helpline:\n"
            "- National Suicide Prevention Lifeline: 988 or 1-800-273-8255\n"
            "- Crisis Text Line: Text HOME to 741741\n\n"
            "Would you like to talk more about what you're experiencing? I'm here to listen."
        )

    def detect_emotion(self, text):
        """Detect primary emotion in text"""
        emotion_keywords = {
            "sadness": ["sad", "depressed", "unhappy", "miserable", "down", "blue", "depressing"],
            "anxiety": ["anxious", "worried", "nervous", "stressed", "panicking", "afraid", "scared", "fear"],
            "anger": ["angry", "mad", "furious", "irritated", "annoyed", "frustrated", "rage"],
            "hopelessness": ["hopeless", "pointless", "worthless", "empty", "numb", "meaningless"],
            "loneliness": ["lonely", "alone", "isolated", "abandoned", "rejected", "nobody", "terrible"]
        }

        text_lower = text.lower()
        emotion_scores = {}

        # Memory-optimized scoring
        for emotion, keywords in emotion_keywords.items():
            score = 0
            for keyword in keywords:
                if keyword in text_lower:
                    score += 1
                    # Early exit optimization - if we found a match, don't check every variant
                    break
            emotion_scores[emotion] = score

        if max(emotion_scores.values()) > 0:
            primary_emotion = max(emotion_scores, key=emotion_scores.get)
            return primary_emotion
        else:
            return "neutral"

    def track_emotion(self, user_id, emotion):
        """Track emotions over time for a user"""
        if user_id not in self.user_emotions:
            self.user_emotions[user_id] = []

        # Add new emotion and limit history to most recent 3 (reduced from 10)
        self.user_emotions[user_id].append(emotion)
        if len(self.user_emotions[user_id]) > 3:
            self.user_emotions[user_id] = self.user_emotions[user_id][-3:]

    def get_emotion_trend(self, user_id):
        """Analyze emotion trends for a user"""
        if user_id not in self.user_emotions or len(self.user_emotions[user_id]) < 2:  # Reduced from 3
            return None

        # Simplified trend analysis
        recent_emotions = self.user_emotions[user_id]

        # Check if emotions are improving
        if recent_emotions[-1] == "neutral" and any(e != "neutral" for e in recent_emotions[:-1]):
            return "improving"

        # Check if same negative emotion persists
        if recent_emotions.count(recent_emotions[0]) == len(recent_emotions) and recent_emotions[0] != "neutral":
            return "persistent_" + recent_emotions[0]

        return None

    def preprocess_input(self, user_input):
        """Format user input with appropriate instruction tags - OPTIMIZED"""
        # Use regex sub only if necessary
        if "[INST]" in user_input or "[/INST]" in user_input:
            clean_input = re.sub(r'\[INST\]|\[/INST\]', '', user_input).strip()
        else:
            clean_input = user_input.strip()
        
        # Format with instruction tags
        return f"<s>[INST] {clean_input} [/INST]"

    def format_conversation_with_history(self, user_id, new_input):
        """Format minimal conversation history for the model input - MEMORY OPTIMIZED"""
        if user_id not in self.last_user_messages:
            return self.preprocess_input(new_input)

        # Extremely minimal history format - just combine previous input with current
        messages = self.last_user_messages[user_id]
        if not messages:
            return self.preprocess_input(new_input)
            
        # Take only the last message as context
        formatted_text = "<s>[INST] "
        
        # Add context from last message
        if len(messages) > 0:
            formatted_text += f"Previous message: {messages[-1]}\n\nCurrent message: {new_input}"
        else:
            formatted_text += new_input
            
        formatted_text += " [/INST]"
        
        return formatted_text

    def postprocess_response(self, response_text):
        """Clean up model response - OPTIMIZED"""
        try:
            # Use more efficient string operations when possible
            parts = response_text.split('[/INST]')
            if len(parts) > 1:
                response = parts[-1].strip()
            else:
                response = response_text.strip()
            
            # Clean only if necessary
            if '<s>' in response or '</s>' in response:
                response = response.replace('<s>', '').replace('</s>', '')
            
            return response
            
        except Exception as e:
            print(f"Error in postprocess_response: {e}")
            # Basic fallback
            return response_text.replace('[/INST]', '').replace('[INST]', '').strip()

    def check_safety(self, user_input):
        """Check if input suggests a crisis situation"""
        input_lower = user_input.lower()
        for keyword in self.crisis_keywords:
            if keyword in input_lower:
                return self.crisis_response
        return None

    def generate_response(self, user_input, user_id="default_user"):
        """Generate an appropriate emotional support response - MEMORY OPTIMIZED"""
        try:
            # Check for crisis signals
            safety_response = self.check_safety(user_input)
            if safety_response:
                return safety_response

            # Detect and track emotion
            emotion = self.detect_emotion(user_input)
            self.track_emotion(user_id, emotion)

            # Format the input with minimal conversation history
            if user_id in self.last_user_messages and len(self.last_user_messages[user_id]) > 0:
                formatted_input = self.format_conversation_with_history(user_id, user_input)
            else:
                formatted_input = self.preprocess_input(user_input)

            # Aggressive memory cleanup before generation
            if torch.cuda.is_available() and not self.is_dummy_mode:
                torch.cuda.empty_cache()
                gc.collect()
            
            # Generate response with memory optimization
            try:
                if self.is_dummy_mode:
                    response = self.text_generation_pipeline(formatted_input)[0]['generated_text']
                else:
                    # Check if input is too long and trim aggressively
                    if len(formatted_input) > 500:  # More aggressive truncation
                        print("Input too long, truncating history...")
                        formatted_input = self.preprocess_input(user_input)
                        
                    with torch.no_grad():
                        try:
                            response = self.text_generation_pipeline(
                                formatted_input,
                                return_full_text=False
                            )[0]['generated_text']
                            
                            # Immediate cleanup after generation
                            if torch.cuda.is_available():
                                torch.cuda.empty_cache()
                                
                        except RuntimeError as e:
                            if "out of memory" in str(e).lower():
                                print("GPU out of memory. Attempting recovery...")
                                gc.collect()
                                torch.cuda.empty_cache()
                                # Try with bare minimum input
                                shortened_input = f"<s>[INST] {user_input} [/INST]"
                                response = self.text_generation_pipeline(
                                    shortened_input,
                                    max_new_tokens=64,  # Reduce output size during OOM
                                    return_full_text=False
                                )[0]['generated_text']
                            else:
                                raise
            except Exception as e:
                print(f"Error in pipeline: {e}")
                response = "I apologize, but I'm having trouble processing your message right now. Could you share a bit more about what you're feeling?"

            # Clean up the response
            clean_response = self.postprocess_response(response)
            
            # Force garbage collection after response generation
            gc.collect()
            if torch.cuda.is_available() and not self.is_dummy_mode:
                torch.cuda.empty_cache()

            return clean_response

        except Exception as e:
            print(f"Error generating response: {e}")
            return "I'm having trouble processing that right now. Could you rephrase or try again later?"

    def chat(self, user_input, user_id="default_user"):
        """Main chat interface with memory-optimized management"""
        try:
            # Initialize history for new users
            if user_id not in self.last_user_messages:
                self.last_user_messages[user_id] = []

            # Clean user input
            cleaned_input = re.sub(r'\[INST\]|\[/INST\]', '', user_input).strip() if '[INST]' in user_input else user_input.strip()
                
            # Generate response using conversation context
            response = self.generate_response(user_input, user_id)

            # MEMORY OPTIMIZATION: Only store user messages, not bot responses
            self.last_user_messages[user_id].append(cleaned_input)
            
            # Keep only the most recent messages
            if len(self.last_user_messages[user_id]) > self.max_history_length:
                self.last_user_messages[user_id] = self.last_user_messages[user_id][-self.max_history_length:]

            # Check for emotional trends and adjust response if needed
            emotion_trend = self.get_emotion_trend(user_id)
            if emotion_trend == "persistent_sadness" or emotion_trend == "persistent_hopelessness":
                response += "\n\nI've noticed that you've been feeling down for a while. Have you considered speaking with a mental health professional who might provide additional support?"

            # Force memory cleanup after response is ready
            gc.collect()
            if torch.cuda.is_available() and not self.is_dummy_mode:
                torch.cuda.empty_cache()

            return response

        except Exception as e:
            print(f"Error in chat method: {e}")
            return "I apologize, but I'm having some technical difficulties. Let's try again."

    def display_debug_info(self, user_id="default_user"):
        """Display debug information with optimized memory usage"""
        print("\n--- Debug Information ---")
        
        # Display system info
        if torch.cuda.is_available() and not self.is_dummy_mode:
            free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
            print(f"GPU: {torch.cuda.get_device_name(0)}")
            print(f"Free GPU memory: {free_memory / 1e9:.2f} GB")
        else:
            print("Running on CPU or in dummy mode")
            
        # Display mode information
        print(f"Running in {'dummy' if self.is_dummy_mode else 'normal'} mode")

        # Display detected emotions
        if user_id in self.user_emotions:
            print(f"Emotion history: {self.user_emotions[user_id]}")

            # Display trend if available
            trend = self.get_emotion_trend(user_id)
            if trend:
                print(f"Emotional trend: {trend}")
            else:
                print("Emotional trend: Not enough data or no clear trend")
        else:
            print("No emotion data available")

        # Display conversation memory stats
        if user_id in self.last_user_messages:
            history_length = len(self.last_user_messages[user_id])
            print(f"User message history: {history_length} messages")

            # Display a sample of recent messages
            if history_length > 0:
                print("\nRecent user messages:")
                start_idx = max(0, history_length - 2)  # Show last 2 messages
                for i in range(start_idx, history_length):
                    content_preview = self.last_user_messages[user_id][i][:50] + "..." if len(self.last_user_messages[user_id][i]) > 50 else self.last_user_messages[user_id][i]
                    print(f"  user: {content_preview}")
        else:
            print("No conversation history available")

        print("------------------------\n")

    def cleanup(self):
        """Release resources to prevent memory leaks - ENHANCED"""
        try:
            if hasattr(self, 'model') and self.model and not self.is_dummy_mode:
                # Completely unload pipeline first
                if hasattr(self, 'text_generation_pipeline'):
                    del self.text_generation_pipeline
                
                # Move model to CPU before deletion
                if hasattr(self.model, 'to') and torch.cuda.is_available():
                    try:
                        self.model.to('cpu')
                    except:
                        pass
                
                # Delete model reference
                del self.model
                self.model = None
                
                # Clear any stored message history
                if hasattr(self, 'last_user_messages'):
                    self.last_user_messages = {}
                if hasattr(self, 'user_emotions'):
                    self.user_emotions = {}
                
                # Force aggressive garbage collection
                gc.collect()
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                
                print("Resources released successfully")
        except Exception as e:
            print(f"Error during cleanup: {e}")


# Function to initialize bot with proper error handling
def initialize_bot(use_dummy_if_failed=True):
    """Initialize EmotionalSupportBot with memory-optimized error handling"""
    try:
        print("Initializing EmotionalSupportBot...")
        # First clean memory
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        # Check available memory
        if torch.cuda.is_available():
            free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
            print(f"Available GPU memory: {free_memory / 1e9:.2f} GB")
            if free_memory < 1e9:  # Less than 1GB free
                print("Warning: Low GPU memory. Consider using CPU mode.")
                
        bot = EmotionalSupportBot(cpu_fallback=use_dummy_if_failed)
        return bot
    except Exception as e:
        print(f"Failed to initialize EmotionalSupportBot: {e}")
        if use_dummy_if_failed:
            print("Attempting to initialize demo bot instead...")
            return initialize_demo_bot()
        return None

# Demo bot function remains as backup
def initialize_demo_bot():
    """Initialize a demo version of the bot for interactive testing"""
    print("Initializing EmotionalSupportBot in demo mode...")

    bot = EmotionalSupportBot()
    bot._initialize_dummy_mode()

    return bot

def run_interactive_session():
    """Run an interactive session with the EmotionalSupportBot"""
    # Clean memory before starting
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        
    # Try to initialize with real model, fall back to demo if needed
    bot = initialize_bot(use_dummy_if_failed=True)

    if not bot:
        print("Could not initialize bot. Exiting.")
        return

    print("\n=== EmotionalSupportBot Interactive Session ===")
    print("Type your messages and the bot will respond.")
    print("Special commands:")
    print("  /debug  - Show debug information about conversation and emotions")
    print("  /reset  - Reset the conversation history")
    print("  /exit   - End the session")
    print("  /clean  - Force memory cleanup")
    print("============================================\n")

    user_id = "interactive_user"

    while True:
        try:
            # Get user input
            user_input = input("You: ").strip()

            # Check for special commands
            if user_input.lower() == "/exit":
                print("Ending session. Take care!")
                # Cleanup before exit
                if hasattr(bot, 'cleanup'):
                    bot.cleanup()
                break
            elif user_input.lower() == "/debug":
                bot.display_debug_info(user_id)
                continue
            elif user_input.lower() == "/clean":
                # Force memory cleanup
                if hasattr(bot, 'cleanup'):
                    bot.cleanup()
                    print("Memory cleaned. Reinitializing...")
                    bot = initialize_bot(use_dummy_if_failed=True)
                    if not bot:
                        print("Failed to reinitialize. Exiting.")
                        break
                continue
            elif user_input.lower() == "/reset":
                if user_id in bot.last_user_messages:
                    bot.last_user_messages[user_id] = []
                if user_id in bot.user_emotions:
                    bot.user_emotions[user_id] = []
                print("Conversation history and emotion tracking have been reset.")
                continue
            elif not user_input:
                continue

            # Process the input and get response
            response = bot.chat(user_input, user_id)

            # Display the response
            print(f"Bot: {response}")

            # Show current emotion after each exchange
            if user_id in bot.user_emotions and bot.user_emotions[user_id]:
                current_emotion = bot.user_emotions[user_id][-1]
                if current_emotion != "neutral":
                    print(f"[Detected emotion: {current_emotion}]")

            # Periodically check memory status
            if torch.cuda.is_available() and hasattr(bot, 'model') and bot.model and not bot.is_dummy_mode:
                try:
                    free_memory = torch.cuda.memory_reserved(0) - torch.cuda.memory_allocated(0)
                    if free_memory < 500e6:  # Less than 500MB free
                        print("[Warning: Low GPU memory. Consider using /clean to free up resources]")
                except:
                    pass  # Ignore errors in memory checking

        except KeyboardInterrupt:
            print("\nSession interrupted. Ending session.")
            if hasattr(bot, 'cleanup'):
                bot.cleanup()
            break
        except Exception as e:
            print(f"Error in interactive session: {e}")
            print("Let's continue anyway.")

# Main execution
if __name__ == "__main__":
    try:
        run_interactive_session()
    finally:
        # Final cleanup before exit
        print("Cleaning up resources...")
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

2025-04-23 10:01:57.007101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745402517.030713     176 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745402517.037448     176 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Initializing EmotionalSupportBot...
Available GPU memory: 0.00 GB
Loading model and tokenizer...
CUDA available: Tesla T4
Total GPU memory: 15.83 GB
Free GPU memory: 0.00 GB
Error loading model: No package metadata was found for bitsandbytes
Attempting alternative model loading approach...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

Bot: Ah, I see! In that case, here are some movies that might help you feel better when you're in a bad mood:

1. Comedy classics like "The Big Lebowski" or "Monty Python and the Holy Grail" can be great for lifting your spirits with their witty humor and lighthearted tone.
2. Uplifting dramas like "Forrest Gump," "The Shawshank Redemption," or "The Pursuit of Happyness" often have inspiring stories and characters that can help shift your perspective on life
[Warning: Low GPU memory. Consider using /clean to free up resources]


You:  Tell me what mood i am in now from my previous messages


Bot: Based on your previous messages, it seems like you are feeling a bit down or melancholic. You mentioned being in a "bad mood" and wanting to read books to improve your emotional state.
[Warning: Low GPU memory. Consider using /clean to free up resources]


You:  /exit


Ending session. Take care!
Resources released successfully
Cleaning up resources...
